In [1]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

In [2]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="neo4j")

In [ ]:
### Seeding the database

In [3]:
graph.query(
    """
MERGE (m:Movie {name:"Top Gun"})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

In [10]:
graph.refresh_schema()


In [4]:
print(graph.schema)

Node properties are the following:
Movie {title: STRING, tagline: STRING, released: INTEGER, name: STRING},Person {born: INTEGER, name: STRING},Actor {name: STRING}
Relationship properties are the following:
ACTED_IN {roles: LIST},REVIEWED {summary: STRING, rating: INTEGER}
The relationships are the following:
(:Person)-[:ACTED_IN]->(:Movie),(:Person)-[:WROTE]->(:Movie),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:PRODUCED]->(:Movie),(:Person)-[:FOLLOWS]->(:Person),(:Person)-[:REVIEWED]->(:Movie),(:Actor)-[:ACTED_IN]->(:Movie)


In [5]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True)

In [6]:
chain.run("Who played in Top Gun?")

/Users/riaz/miniconda3/envs/interview/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: 'Top Gun'})
RETURN p.name
Full Context:
[{'p.name': 'Val Kilmer'}, {'p.name': 'Meg Ryan'}, {'p.name': 'Tom Skerritt'}, {'p.name': 'Kelly McGillis'}, {'p.name': 'Tom Cruise'}, {'p.name': 'Anthony Edwards'}]

> Finished chain.


'The actors who played in Top Gun are Val Kilmer, Meg Ryan, Tom Skerritt, Kelly McGillis, Tom Cruise, and Anthony Edwards.'

In [7]:
### returning the result directly.

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_direct=True
)

In [8]:
chain.run("Who played in Top Gun?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: 'Top Gun'})
RETURN p.name

> Finished chain.


[{'p.name': 'Val Kilmer'},
 {'p.name': 'Meg Ryan'},
 {'p.name': 'Tom Skerritt'},
 {'p.name': 'Kelly McGillis'},
 {'p.name': 'Tom Cruise'},
 {'p.name': 'Anthony Edwards'}]

: 